Início

In [1]:
#dfp são consultas com o ano inteiro
import pandas as pd
import requests
import os
import zipfile

In [2]:
diretorio_atual = os.getcwd()
if diretorio_atual == 'd:\OneDrive\Documentos\.TCC USP\PyIbov\dados_cvm\DFP':
    pass
else:
    os.chdir(f"{diretorio_atual}/dados_cvm/DFP")

Importação de bases

In [3]:
anos = range(2018,2023)
url_base = 'https://dados.cvm.gov.br/dados/cia_aberta/doc/DFP/DADOS/'

for ano in anos:
    download = requests.get(url_base+f'dfp_cia_aberta_{ano}.zip')

    open(f'dfp_cia_aberta_{ano}.zip', 'wb').write(download.content)

try:
    os.remove('filtro_dreDFP.csv')
except:
    pass

print(pd.Timestamp.now())

2022-09-28 18:05:42.470401


Construção e formatação da base de dados

In [4]:
pd.Timestamp.now()
lista_demonstracoes = []
diretorio_atual = os.getcwd()
for arquivo in os.listdir(diretorio_atual):
    if arquivo == '.ipynb_checkpoints':
        pass
    else:
        arquivo_zip = zipfile.ZipFile(arquivo)

        for planilha in arquivo_zip.namelist():
            demonstracao = pd.read_csv(arquivo_zip.open(planilha),sep=';', encoding='ISO-8859-1',
                                        dtype={'ORDEM_EXERC':'category'})
            lista_demonstracoes.append(demonstracao)
pd.Timestamp.now()

Timestamp('2022-09-28 18:06:01.693402')

In [5]:
base_dados = pd.concat(lista_demonstracoes)
base_dados[:5]
pd.Timestamp.now()

Timestamp('2022-09-28 18:06:07.939404')

In [6]:
base_dados[['con_ind','tipo_dem']] = base_dados['GRUPO_DFP'].str.split('-',expand=True)
pd.Timestamp.now()

Timestamp('2022-09-28 18:06:23.450406')

In [7]:
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-28 18:06:46.729403')

In [8]:
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()
pd.Timestamp.now()

Timestamp('2022-09-28 18:06:52.921403')

In [9]:
base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']
pd.Timestamp.now()

Timestamp('2022-09-28 18:07:07.723403')

In [10]:
lista_dem = base_dados['tipo_dem'].unique()
#base_dados['tipo_dem'].unique()
print(lista_dem)
pd.Timestamp.now()

[nan 'Balanço Patrimonial Ativo' 'Balanço Patrimonial Passivo'
 'Demonstração do Fluxo de Caixa (Método Direto)'
 'Demonstração do Fluxo de Caixa (Método Indireto)'
 'Demonstração das Mutações do Patrimônio Líquido'
 'Demonstração de Resultado Abrangente' 'Demonstração do Resultado'
 'Demonstração de Valor Adicionado']


Timestamp('2022-09-28 18:07:09.033163')

In [11]:
lista_empresas = base_dados['DENOM_CIA'].unique()
print(lista_empresas)
pd.Timestamp.now()

['BCO BRASIL S.A.' 'BRB BCO DE BRASILIA S.A.'
 'CENTRAIS ELET BRAS S.A. - ELETROBRAS'
 'COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB' 'SHOPPING CENTER TACARUNA SA'
 'NEUMARKT TRADE AND FINANCIAL CENTER S/A' 'CIMS S.A.'
 'TELEC BRASILEIRAS S.A. TELEBRAS'
 'KOSMOS COMÉRCIO DE VESTUÁRIO S/A - EM RECUPERAÇÃO JUDICIAL'
 'ATOM EMPREENDIMENTOS E PARTICIPAÇÕES S.A.'
 'BNDES PARTICIPACOES S.A. - BNDESPAR'
 'LM TRANSPORTES INTERESTADUAIS SERVIÇOS E COMÉRCIO S.A.'
 'BANCO INTER S.A.' 'ELETRON S.A.' 'SERRA AZUL WATER PARK SA'
 'STATKRAFT ENERGIAS RENOVAVEIS S.A.' 'INVESTCO S.A.'
 'LITEL PARTICIPACOES S.A.' 'AMERICANAS S.A.'
 'CONC RODOVIA PRES. DUTRA S.A.' 'ENERGISA S.A.'
 'SANTA CATARINA PART INVEST SA' 'FERROVIA CENTRO-ATLANTICA S.A.'
 'CONC RIO-TERESOPOLIS S.A.' 'LINHA AMARELA S.A. - LAMSA'
 'CAIXA ADM DIV PUB ESTADUAL SA' 'CIELO S.A. -  INSTITUIÇÃO DE PAGAMENTO'
 'NEOENERGIA S.A.' 'ELETROBRÁS PARTICIPAÇÕES S.A. - ELETROPAR'
 'BBM LOGÍSTICA S.A.' 'ITAITINGA PARTICIPACOES S.A.' 'RUMO MALHA SUL S.A.'
 

Timestamp('2022-09-28 18:07:15.763161')

In [12]:
lista_conta = base_dados['DS_CONTA'].unique()
print(lista_conta)
pd.Timestamp.now()

[nan 'Ativo Total' 'Caixa e Equivalentes de Caixa' ...
 'Demais despesas financeiras'
 'Ganho líquido decorrente de mudança no valor justo e realização da mais ou menos valia dos ativos bi'
 '41689']


Timestamp('2022-09-28 18:07:16.042164')

Filtro para empresas desejadas

In [13]:
filtro = ['IRB - BRASIL RESSEGUROS S.A.']
#'BCO BRASIL S.A.', 'BRB BCO DE BRASILIA S.A.',
filtro_dre = base_dados[base_dados.DENOM_CIA.isin(filtro)
                        #&(base_dados['tipo_dem']=='Demonstração de Resultados')
                        ]

In [17]:
lista_conta = filtro_dre['DS_CONTA'].unique()
lista_dem = filtro_dre['tipo_dem'].unique()


In [21]:
filtro_dre['base'] = 'dfp'
filtro_dre

C:\Users\olive\AppData\Local\Temp\ipykernel_22900\3822054259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtro_dre['base'] = 'dfp'


,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,...,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF,con_ind,tipo_dem,base
608,33.376.989/0001-91,2018-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,DFP,80516.0,2019-02-07,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dfp
609,33.376.989/0001-91,2018-12-31,2,IRB - BRASIL RESSEGUROS S.A.,24180,DFP,81058.0,2019-03-07,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dfp
610,33.376.989/0001-91,2018-12-31,3,IRB - BRASIL RESSEGUROS S.A.,24180,DFP,81059.0,2019-03-07,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dfp
611,33.376.989/0001-91,2018-12-31,4,IRB - BRASIL RESSEGUROS S.A.,24180,DFP,81090.0,2019-03-08,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dfp
26137,33.376.989/0001-91,2018-12-31,4,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Consolidado - Balanço Patrimonial Ativo,...,2018-12-31,1,Ativo Total,15940434.0,S,NaN,NaN,DF Consolidado,Balanço Patrimonial Ativo,dfp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42395,33.376.989/0001-91,2021-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,2021-12-31,7.11.04.01,Juros sobre o Capital Próprio,0.0,S,2021-01-01,NaN,DF Individual,Demonstração de Valor Adicionado,dfp
42397,33.376.989/0001-91,2021-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,2021-12-31,7.11.04.02,Dividendos,0.0,S,2021-01-01,NaN,DF Individual,Demonstração de Valor Adicionado,dfp
42399,33.376.989/0001-91,2021-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,2021-12-31,7.11.04.03,Lucros Retidos / Prejuízo do Período,-682701.0,S,2021-01-01,NaN,DF Individual,Demonstração de Valor Adicionado,dfp
42401,33.376.989/0001-91,2021-12-31,1,IRB - BRASIL RESSEGUROS S.A.,24180,NaN,NaN,NaN,NaN,DF Individual - Demonstração de Valor Adicionado,...,2021-12-31,7.11.05,Outros,851374.0,S,2021-01-01,NaN,DF Individual,Demonstração de Valor Adicionado,dfp


In [22]:
campos = ['Imposto de Renda e Contribuição Social sobre o Lucro'
            ,'Receitas das Operações'
            ,'Lucro Líquido do Período'
            ,'Lucro Líquido Consolidado do Período'
            ,'Lucro/Prejuízo Consolidado do Período'
            ,'Lucro/Prejuízo do Período'
            ,'Imposto de Renda e Contribuição Social sobre o Lucro'
            ,'Depreciação, Amortização e Exaustão']
filtro_dre2 = filtro_dre[filtro_dre.DS_CONTA.isin(campos)]

In [23]:
filtro_dre.to_csv('filtro_dreDFP.csv',sep=';')
pd.Timestamp.now()


Timestamp('2022-09-28 23:36:38.455521')